# Collaborative Recommendations Engine (Part 1/3)

> This notebook is part of a series of notebooks that will walk you through the process of building a good collaborative recommendations engine (while also including our mistakes that we did). The series is broken up into three parts:

- **Part 1: Our Attempt at Building an Item-Item Collaborative Recommendations Engine**
- Part 2: Fixing our Item-Item Collaborative Recommendations Engine
- Part 3: Improving our Collaborative Recommendations Engine by leveraging other techniques than Item-Item Collaborative Filtering...

## Part 1: Our Attempt at Building an Item-Item Collaborative Recommendations Engine
When we started on our recommender system, we would have never expected that we would have to go through so many iterations to get to a working model. We started off with a simple item-item collaborative filtering model (based on the slides), but we quickly realized that it's harder than it look like to build a model that is scalable...

Throughout the process, we will be using Movies Dataset which consists of X ratings from Y users on Z movies (where the row format is: `movieId,userId,rating,timestamp`). The endgoal is to be able to use our Kaggle Dataset of 100k ratings, but to start off, we will be using a smaller homemade dataset based on the slides to validate our methods.

Lastly, it's important to mentionned that we will be using a sparse utility matrix to store our ratings. We won't have to collect the data in the utility matrix, since it's already provided by the dataset. Ratings vary from 0 to 5, and we will be using a 0 to represent a missing rating. Our endgoal is to be able to extrapolate unknown ratings from the known ones, while also evaluate these extrapolate methods to measure the success/performance of our system.

More details about our techniques will be provided throughout the notebook, but for now let's start!

### Step 1: Importing the necessary libraries


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, sum as spark_sum
from pyspark.sql.types import DoubleType, FloatType
from pyspark.sql import SparkSession

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import  Vectors
from pyspark.ml.linalg import Vectors

### Step 2: Spark Configuration/Setup
We will be using Spark to build our recommender system. Spark is a distributed computing framework that allows us to run our code on a cluster of machines. It's a great tool to use when we want to scale our code to a large dataset. We will be using the `pyspark` library to run our code on a local machine. We will be using the `pyspark.sql` library to create our dataframes, and the `pyspark.ml` library to build our recommender system.

Throughout the program, we will also need to use udf-functions to create our own functions. They are implemented and registered in the following code-block.

In [8]:
# Spark Initialization/Setup

spark = SparkSession.builder \
    .appName("recommenderTest") \
    .config("spark.some.config.option", "some-value") \
    .config("spark.executor.memory", "7g") \
    .config("spark.driver.memory", "7g") \
    .config("spark.sql.shuffle.partitions", "32") \
    .config("spark.sql.pivotMaxValues", "20000") \
    .config("spark.master", "local[*]") \
    .config("spark.sql.codegen.wholeStage", "false") \
    .getOrCreate()

spark.sparkContext.setLogLevel("FATAL")


# -------------- UDFs Helper's Functions Implementation --------------

# Pearson Correlation Average Calculation (for Rx' and Ry')
# Note: Our average function is not scalable, but it is good enough for our purposes for now, because
# we won't move forward with it when proceeding with larger datasets (more details soon...)
def pearson_average(v):
    sum_nonzero = sum(v)
    count_nonzero = len([e for e in v if e != 0])
    mean = sum_nonzero / count_nonzero
    v2 = [e - mean if e != 0 else 0 for e in v] # now subtract the mean from each non zero element
    return Vectors.dense(v2)

# Pearson Correlation Coefficient Calculation
# Note: Although the correlation average calculation isn't scalable, the dot product/norm calculation is.
def co_sym (x, y):
    pearson1 = pearson_average(x)
    pearson2 = pearson_average(y)
    return float(pearson1.dot(pearson2)/(Vectors.norm(pearson1,2)*Vectors.norm(pearson2,2)))

# Matrix Dot Product Calculation: (v1 * v2) / sum(v1)
def dot_product_divided_by_sum(v1, v2):
    dot_product = float(v1.dot(v2))
    sum_v1 = float(v1.sum())
    if sum_v1 == 0:
        return 0.0
    return dot_product/sum_v1

# -------------- UDFs Initialization --------------
dot_udf = udf(co_sym, DoubleType())
spark.udf.register("dot_udf", dot_udf)
spark.udf.register("dot_product_divided_by_sum", dot_product_divided_by_sum)

<function __main__.dot_product_divided_by_sum(v1, v2)>

### Step 3: Data Preparation/Loading
Unlike content-based, we don't need to do any data preparation for collaborative filtering (except removing timestamp since it will not be used). We will be directly using the `ratings.csv` file (format: userId, movieId, rating, timestamp).

For this notebook, we will be using ratings_tiny.csv to validate our method. Ratings_tiny.csv is the same as the example in the slides (p.27 in Collaborative-Slideset)

In [9]:
df = spark.read.csv("data/ratings_tiny.csv", header=True, inferSchema=True)
df = df.drop("timestamp")


# -------------- Data Initialization --------------
# DF to be used for User-Movie-Rating Matrix (will be used later on - Setting initial state for now...)
df_user_movie_rating = df
# DF to be used for Tentative 2... (will be used later on - Setting initial state for now...)
df2 = df_user_movie_rating.groupBy("userId").pivot("movieId").agg({"rating": "first"}).fillna(0)

### Step 4: Data Modeling
This step is crucial in order to build our recommender. This is where we will be building our item-item collaborative filtering model. We will be using the following steps:
1. Create Utility Matrix (Item-Item) where the rows are the movies and the columns are the users (in our code: this is represented as df_user_movie_rating)
2. Calculate the similarity between each pair of movies (in our code: this is represented as similarity_matrix) using the following formula (Pearson Correlation Coefficient):
$$similarity = \frac{\sum_{u \in U} (r_{u,i} - \bar{r_i})(r_{u,j} - \bar{r_j})}{\sqrt{\sum_{u \in U} (r_{u,i} - \bar{r_i})^2} \sqrt{\sum_{u \in U} (r_{u,j} - \bar{r_j})^2}}$$
3. Build a new matrix which will contain all the movies combinations (similarity) for each user. This matrix will be used to predict the ratings for each user

**Note**: In this notebook, we won't split our dataset into training and testing set. We will be using the whole dataset to build our model in order to validate our model with the example in the slides. In the next notebook, we will be splitting our dataset into training and testing set.

In [10]:
# -- Building Similarity Matrix --
df = df.groupBy("movieId").pivot("userId").agg({"rating": "first"}).fillna(0)

# Build Vector Columns from DF Matrix
assembler = VectorAssembler(inputCols=df.columns[1:], outputCol="features")
df_vector = assembler.transform(df).select('movieId', 'features')
df_vector = df_vector.repartition(10)

# Compute Pearson Correlation to fill data into Similarity Matrix
similarity_matrix = df_vector.alias("a").crossJoin(df_vector.alias("b")) \
    .where("a.movieId != b.movieId") \
    .selectExpr("a.movieId as movieId", "b.movieId as movieId_1",
                "dot_udf(a.features, b.features) as similarity")
similarity_matrix.show(10, 10)

# Build User-Movie-Rating Matrix (where for each user, we have all the movies combinations with the similarity values)
df_user_movie_rating = df_user_movie_rating.join(similarity_matrix, df_user_movie_rating.movieId == similarity_matrix.movieId, how='left').drop(similarity_matrix.movieId)
df_user_movie_rating = df_user_movie_rating.withColumnRenamed("similarity.movieId", "movie2")
df_user_movie_rating.show(10,10)

+-------+---------+----------+
|movieId|movieId_1|similarity|
+-------+---------+----------+
|      4|        3|-0.6239...|
|      4|        6|-0.2353...|
|      4|        5|0.45873...|
|      4|        1|-0.1024...|
|      4|        2|0.46800...|
|      3|        4|-0.6239...|
|      3|        6|0.50636...|
|      3|        5|-0.2842...|
|      3|        1|0.41403...|
|      3|        2|-0.5262...|
+-------+---------+----------+
only showing top 10 rows

+-------+------+------+---------+----------+
|movieId|userId|rating|movieId_1|similarity|
+-------+------+------+---------+----------+
|      1|     1|     1|        2|-0.1785...|
|      1|     1|     1|        5|-0.3089...|
|      1|     1|     1|        6|0.58703...|
|      1|     1|     1|        3|0.41403...|
|      1|     1|     1|        4|-0.1024...|
|      1|     3|     3|        2|-0.1785...|
|      1|     3|     3|        5|-0.3089...|
|      1|     3|     3|        6|0.58703...|
|      1|     3|     3|        3|0.41403...|


### Step 5: Data Prediction
Now that we have a dataframe containing all the possible movies similarity combinations for each users, we can use this dataframe **to attempt** to predict the ratings for each user. We need to be able to predict ratings in order to be able to evaluate our model.

In [11]:
# -- Predicting User Rating Function where you need to pass a userId, movieId + dataframe --
def predict_user_rating(user_id, movie_id, smDF): # Can't register a UDF with a DF as input
    
    # Filter similarity matrix to include only ratings for the given user and similar movies
    user_ratings = smDF.filter((col("userId") == user_id) & (col("movieId_1") == movie_id))
    
    # Sort the ratings by similarity in descending order and select the top 2 most similar movies
    user_ratings = user_ratings.sort(col("similarity").desc()).limit(2)
    
    # Calculate the predicted rating by computing a weighted average of the user's ratings for similar movies
    user_ratings = user_ratings.withColumn("weighted_rating", (col("rating") * col("similarity")).cast(FloatType()))
    numerator = user_ratings.agg(spark_sum("weighted_rating")).collect()[0][0]
    denominator = user_ratings.agg(spark_sum("similarity")).collect()[0][0]
    
    if denominator != 0:
        predicted_rating = numerator / denominator
    else:
        predicted_rating = None
    
    return predicted_rating

# Calling Function to Predict Rating for User Id = 5 with Movie Id = 1
predicted_rating = predict_user_rating(user_id=5, movie_id=1, smDF=df_user_movie_rating)
print("Predicted Rating for User Id = 5 with Movie Id = 1 is: ", predicted_rating)


Predicted Rating for User Id = 5 with Movie Id = 1 is:  2.5864068884261053


If you have a look at the slides (at page 31), you can indeed see that we were able to predict the ratings exactly as we did in the slides. This is a good sign that our model is working as expected.

<img src="./img/pearsonPrediction.png" width="300">

**However**, being able to predict one rating is not enough. In reality, our test set would consist of multiple movies to predict. We can't just call this method one-at-a-time because it's simply not scalable. 

This is where we started to get a lot of issues with our logic! By the way our method was implemented, we can't register this method as a udf because we are passing a Dataframe as parameter. After trying a LOT of different ways to make it work in order to be able to register it, we decided to move on with another techniques to be able to predict multiple ratings in parallel.

#### Attempt to predict multiple ratings with another techniques...
This is a tentative to predict multiple ratings in parallel (**Spoiler Alert**: it did not work...) We tried fixing it with these following steps:
1. Take our Similarity Matrix and Pivot it so that we have a matrix containing all the similarity combinations (# of Movies x # of Movies)
2. From there, we would have a sparse matrix where we would have to fill the missing values by multiplying all the similarity of a row with the rating of the users. We would then be able to use this matrix to predict the ratings for each user (in theory)

However, in reality, it wasn't scalable... Our code was working and producing the right results, but due to the multitude of operations, it was simply too slow for 100k ratings.

In the end, it was actually a good thing to move away from this technique because of these issues:
- It was not scalable
- It's not efficient to predict EVERY values (including the ones that are not in the test set)
- We can't decide the number of items most similar to x. We need to multiply all of them, which is not efficient. We need to find a good in-between N-value.

In [12]:
# Pivot the similarity matrix to get a matrix with movieId as rows and movieId_1 as columns
similarity_matrix = similarity_matrix.groupBy("movieId").pivot("movieId_1").agg({"similarity": "first"}).fillna(0)
similarity_matrix = similarity_matrix.sort("movieId")
similarity_matrix.select(similarity_matrix.columns[:10]).show(truncate=False)

# Build Vector Columns From Similarity Matrix
assembler = VectorAssembler(inputCols=similarity_matrix.columns[1:], outputCol="features")
similarity_matrix = assembler.transform(similarity_matrix).select('movieId', 'features')

# Build Vector Columns From User-Movie-Rating Matrix
assembler = VectorAssembler(inputCols=df2.columns[1:], outputCol="features")
df2 = assembler.transform(df2).select('userId', 'features')

# Compute the recommendation matrix (expensive...)
recommendation_matrix = similarity_matrix.alias("a").crossJoin(df2.alias("b")) \
    .selectExpr("a.movieId as movieId", "b.userId as userId",
                "dot_product_divided_by_sum(a.features, b.features) as recommendation").select("userId", "movieId", "recommendation")

recommendation_matrix.show(10)

spark.stop() # Close Spark Session

+-------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+
|movieId|1                   |2                   |3                  |4                   |5                   |6                   |
+-------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+
|1      |0.0                 |-0.17854212213729673|0.41403933560541256|-0.10245014273309601|-0.30895719032666236|0.5870395085642741  |
|2      |-0.17854212213729673|0.0                 |-0.5262348115842176|0.46800784077976626 |0.39891071573694176 |-0.3064397582621859 |
|3      |0.41403933560541256 |-0.5262348115842176 |0.0                |-0.6239806502223061 |-0.2842676218074806 |0.5063696835418333  |
|4      |-0.10245014273309601|0.46800784077976626 |-0.6239806502223061|0.0                 |0.45873490213598356 |-0.2353393621658208 |
|5      |-0.30895719032666236|0.39891071573694176 |-0.2

### Step 6: Now What?
As of now, we are not able to predict multiple values effienctly other than using a for loop. For this reason, we need to look at another techniques, otherwise we won't be able to evaluate our model... More details will follow in the next notebook!
